# Notebook 3: CPUs-GPUs

El entrenamiento de redes es un proceso muy costoso, debido a que tiene que procesar mucha información. En este notebook vamos a realizar una comparación de los entrenamientos de varias redes en distintos entornos, unos contarán solo con su CPU, mientras que otros podrán hacer uso de gráficas y por lo tanto de su correspondiente GPU.

## Pasos previos
Este primero notebook se realiza sobre un ordenador portátil sin gráfica, pero para ver mejor sus características empezaremos mostrando la información del entorno en el que vamos a trabajar.

In [1]:
!apt-get  install lshw
!lshw

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libkmod2 libpci3 libusb-1.0-0 pciutils usbutils
The following NEW packages will be installed:
  libkmod2 libpci3 libusb-1.0-0 lshw pciutils usbutils
0 upgraded, 6 newly installed, 0 to remove and 0 not upgraded.
Need to get 793 kB of archives.
After this operation, 3,054 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu artful/main amd64 libkmod2 amd64 24-1ubuntu2 [40.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu artful/main amd64 libpci3 amd64 1:3.5.2-1ubuntu1 [24.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu artful/main amd64 libusb-1.0-0 amd64 2:1.0.21-2 [43.3 kB]
Get:4 http://archive.ubuntu.com/ubuntu artful-updates/main amd64 lshw amd64 02.18-0.1ubuntu4.1 [231 kB]
Get:5 http://archive.ubuntu.com/ubuntu artful/main amd64 pciutils amd64 1:3.5.2-1ubuntu1 [247 kB]
Get:6 http://archive.ubuntu.com/ubuntu artfu

De todas sus características la que más nos interesa para este trabajo es la información relativa a la GPU, donde podemos destacar:
* Es un Intel(R) Core(TM) i5-4288U CPU @ 2.60GHz.
* Tiene un procesador con un core.
* Anchura: 64 bits.

## Descarga de librerías

El siguiente paso será descargar las librerías necesarias. Para ello descargamos el zip correspondiente y lo descomprimimos.

In [2]:
!wget www.unirioja.es/cu/joheras/redes.zip
!unzip redes.zip

--2018-05-04 16:15:06--  http://www.unirioja.es/cu/joheras/redes.zip
Resolving www.unirioja.es (www.unirioja.es)... 193.144.2.30
Connecting to www.unirioja.es (www.unirioja.es)|193.144.2.30|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5145 (5.0K) [application/zip]
Saving to: ‘redes.zip’

redes.zip           100%[===================>]   5.02K  31.0KB/s    in 0.2s    

2018-05-04 16:15:07 (31.0 KB/s) - ‘redes.zip’ saved [5145/5145]

Archive:  redes.zip
   creating: redes/
  inflating: redes/lenet.py          
  inflating: redes/shallownet.py     
  inflating: redes/resnet.py         
  inflating: redes/vggnet.py         
  inflating: redes/googlenet.py      
  inflating: redes/__init__.py       


Importas los paquetes necesarios, en este caso con las que más vamos a trabajar son:
* En primer lugar importamos las redes con las que vamos a trabajar que son: *LeNet*, *GoogleNet*, *VGGNet*, *ResNet*, *ShallowNet*.
* Keras es una biblioteca de aprendizaje profundo que utiliza por debajo TensorFlow y que nos permitirá trabajar con las redes.
* Sklearn clase de la que heredan todos los algortimos
* Numpy es una extensión de Python, que le permite realizar operaciones con vectores y matrices.
* Matpltolib es una biblioteca para la generación de gráficos a partir de datos.

In [3]:
from redes import *
from keras.optimizers import RMSprop
from keras.optimizers import SGD
from IPython.display import Image
from keras.utils import plot_model
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.datasets import cifar10
from keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

Using TensorFlow backend.


Para entrenar estas redes vamos a trabajar con un dataset ya creado, es el [Cifar-10](https://www.cs.toronto.edu/~kriz/cifar.html), que consta de 60000 imágenes dividido en 10 clases. El dataset se divide en cinco conjuntos de entrenamiento y uno de test, cada uno con 10000 imágenes.

In [4]:
print("[INFO] loading CIFAR-10 data...")
((trainX, trainY), (testX, testY)) = cifar10.load_data()
trainX = trainX.astype("float")
testX = testX.astype("float")

[INFO] loading CIFAR-10 data...
143138816/170498071 [========================>.....] - ETA: 6s

170500096/170498071 [==============================] - 40s 0us/step


El siguiente paso consistirá en preprocesar los datos, es decir, calcularemos la media del dataset de entrenamiento y de test.

In [0]:
mean = np.mean(trainX, axis=0)
trainX -= mean
testX -= mean


Aplicamos las transformaciones correspondientes al conjunto de entrenamiento convirtiendo las etiquestas para procesarlas y luego aplicamos la misma transformación al conjunto de test.

In [0]:
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

Una vez que ya tenemos todo preaparado empezamos a trabajar con los redes.

# LeNet

La arquitectura LeNet es sencilla y pequeña, por lo que es perfecta para enseñar los conceptos básicos de CNN. Se usa principalmente para OCR y reconocimiento de caracteres en documentos.

Definimos la red

In [0]:
model = LeNet.build(width=32, height=32, depth=3, classes=10)

Mostrar arquitectura de la red.

In [8]:
!apt-get install graphviz
!pip install graphviz pydotplus
!pip install --upgrade --force-reinstall pydot
plot_model(model, to_file="lenet.png", show_shapes=True)
Image("lenet.png")

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fontconfig libcairo2 libcdt5 libcgraph6 libdatrie1 libgd3 libgraphite2-3
  libgvc6 libgvpr2 libharfbuzz0b libjbig0 libltdl7 libpango-1.0-0
  libpangocairo-1.0-0 libpangoft2-1.0-0 libpathplan4 libpixman-1-0
  libthai-data libthai0 libtiff5 libwebp6 libxaw7 libxcb-render0 libxcb-shm0
  libxmu6 libxpm4 libxt6
Suggested packages:
  gsfonts graphviz-doc libgd-tools
The following NEW packages will be installed:
  fontconfig graphviz libcairo2 libcdt5 libcgraph6 libdatrie1 libgd3
  libgraphite2-3 libgvc6 libgvpr2 libharfbuzz0b libjbig0 libltdl7
  libpango-1.0-0 libpangocairo-1.0-0 libpangoft2-1.0-0 libpathplan4
  libpixman-1-0 libthai-data libthai0 libtiff5 libwebp6 libxaw7 libxcb-render0
  libxcb-shm0 libxmu6 libxpm4 libxt6
0 upgraded, 28 newly installed, 0 to remove and 0 not upgraded.
Need to get 4,122 kB of archives.
After this operation, 2

Selecting previously unselected package libxcb-shm0:amd64.
Preparing to unpack .../10-libxcb-shm0_1.12-1ubuntu1_amd64.deb ...
Unpacking libxcb-shm0:amd64 (1.12-1ubuntu1) ...
Selecting previously unselected package libcairo2:amd64.
Preparing to unpack .../11-libcairo2_1.14.10-1ubuntu1_amd64.deb ...
Unpacking libcairo2:amd64 (1.14.10-1ubuntu1) ...
Selecting previously unselected package libltdl7:amd64.
Preparing to unpack .../12-libltdl7_2.4.6-2_amd64.deb ...
Unpacking libltdl7:amd64 (2.4.6-2) ...
Selecting previously unselected package libthai-data.
Preparing to unpack .../13-libthai-data_0.1.26-3_all.deb ...
Unpacking libthai-data (0.1.26-3) ...
Selecting previously unselected package libdatrie1:amd64.
Preparing to unpack .../14-libdatrie1_0.2.10-5_amd64.deb ...
Unpacking libdatrie1:amd64 (0.2.10-5) ...
Selecting previously unselected package libthai0:amd64.
Preparing to unpack .../15-libthai0_0.1.26-3_amd64.deb ...
Unpacking libthai0:amd64 (0.1.26-3) ...
Selecting previously unselecte

    100% |████████████████████████████████| 286kB 11.4MB/s 
  Running setup.py bdist_wheel for pydotplus ... - done
  Stored in directory: /content/.cache/pip/wheels/35/7b/ab/66fb7b2ac1f6df87475b09dc48e707b6e0de80a6d8444e3628
Successfully built pydotplus
    100% |████████████████████████████████| 133kB 6.1MB/s 
    100% |████████████████████████████████| 61kB 6.7MB/s 
  Running setup.py bdist_wheel for pydot ... - done
  Stored in directory: /content/.cache/pip/wheels/6a/a5/14/25541ebcdeaf97a37b6d05c7ff15f5bd20f5e91b99d313e5b4
Successfully built pydot
  Found existing installation: pyparsing 2.2.0
    Uninstalling pyparsing-2.2.0:
      Successfully uninstalled pyparsing-2.2.0


ImportError: ignored

Compilar la red.

In [0]:
opt = SGD(lr=0.01)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

Entrenamos la red. 

In [10]:
%%time
print("[INFO] training network...")
H = model.fit(trainX, trainY, validation_data=(testX, testY),batch_size=128, epochs=1, verbose=1)

[INFO] training network...
Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 10s 201us/step - loss: 4.0082 - acc: 0.2783 - val_loss: 1.7272 - val_acc: 0.3751
CPU times: user 6.85 s, sys: 3.37 s, total: 10.2 s
Wall time: 10.3 s


Evaluar la red en el conjunto de test, y mostrar el tiempo. 

In [11]:
%%time
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=128)
print(classification_report(testY.argmax(axis=1),predictions.argmax(axis=1)))

[INFO] evaluating network...
             precision    recall  f1-score   support

          0       0.40      0.63      0.49      1000
          1       0.44      0.47      0.46      1000
          2       0.27      0.34      0.30      1000
          3       0.26      0.17      0.20      1000
          4       0.27      0.27      0.27      1000
          5       0.33      0.27      0.30      1000
          6       0.43      0.18      0.26      1000
          7       0.43      0.39      0.41      1000
          8       0.43      0.55      0.48      1000
          9       0.46      0.47      0.47      1000

avg / total       0.37      0.38      0.36     10000

CPU times: user 410 ms, sys: 100 ms, total: 510 ms
Wall time: 624 ms


# GoogleNet
Google generó la red GoogleNet con más capas (22 capas) internas y más sencillas de computar.  

Definimos la red

In [0]:
model = GoogLeNet.build(width=32, height=32, depth=3, classes=10)

Compilamos la red.

In [0]:
model.compile(loss="categorical_crossentropy", optimizer=RMSprop())


Entrenamos la red.

In [14]:
%%time
H = model.fit(trainX, trainY, validation_data=(testX, testY),batch_size=128, epochs=1, verbose=1)

Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 104s 2ms/step - loss: 1.3015 - val_loss: 1.2627
CPU times: user 1min 15s, sys: 34.3 s, total: 1min 49s
Wall time: 1min 46s


Evaluar la red en el conjunto de test, y mostramos el tiempo que le ha costado.

In [15]:
%%time
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1),predictions.argmax(axis=1)))

[INFO] evaluating network...
             precision    recall  f1-score   support

          0       0.71      0.55      0.62      1000
          1       0.90      0.31      0.47      1000
          2       0.67      0.27      0.38      1000
          3       0.65      0.17      0.27      1000
          4       0.47      0.72      0.57      1000
          5       0.68      0.46      0.55      1000
          6       0.46      0.88      0.60      1000
          7       0.67      0.74      0.70      1000
          8       0.78      0.77      0.77      1000
          9       0.47      0.95      0.63      1000

avg / total       0.65      0.58      0.56     10000

CPU times: user 2.59 s, sys: 889 ms, total: 3.48 s
Wall time: 7.63 s


# VGGNet
Esta red se caracteriza por su simplicidad, utilizando solo capas 3 × 3 convolucionales apiladas una sobre la otra en profundidad creciente.

Definimos la red

In [0]:
model = VGGNet.build(width=32, height=32, depth=3, classes=10)

Compilamos la red.

In [0]:
opt = SGD(lr=0.01, decay=0.01 / 40, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy" , optimizer=opt, metrics=["accuracy" ])

Entrenamos la red. 

In [18]:
%%time
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=1, verbose=1)

Train on 50000 samples, validate on 10000 samples
Epoch 1/1
37952/50000 [=====================>........] - ETA: 6s - loss: 1.6862 - acc: 0.4405

50000/50000 [==============================] - 30s 594us/step - loss: 1.6042 - acc: 0.4625 - val_loss: 1.0818 - val_acc: 0.6203
CPU times: user 27.8 s, sys: 10 s, total: 37.8 s
Wall time: 30.5 s


Evaluar la red en el conjunto de test, y mostramos el tiempo que le ha costado. 

In [19]:
%%time
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1),predictions.argmax(axis=1)))

[INFO] evaluating network...
             precision    recall  f1-score   support

          0       0.71      0.55      0.62      1000
          1       0.75      0.79      0.77      1000
          2       0.54      0.35      0.42      1000
          3       0.44      0.45      0.44      1000
          4       0.53      0.54      0.53      1000
          5       0.53      0.51      0.52      1000
          6       0.65      0.79      0.71      1000
          7       0.65      0.74      0.69      1000
          8       0.65      0.83      0.73      1000
          9       0.72      0.66      0.69      1000

avg / total       0.62      0.62      0.61     10000

CPU times: user 1.28 s, sys: 260 ms, total: 1.54 s
Wall time: 1.8 s


# ResNet

Definimos la red

In [0]:
model = ResNet.build(width=32, height=32, depth=3, classes=10)

Compilamos la red

In [0]:
opt = SGD(lr=0.005)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

In [22]:
%%time
print("[INFO] training network...")
H = model.fit(trainX, trainY, validation_data=(testX, testY),batch_size=128, epochs=1, verbose=1)

[INFO] training network...
Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 214s 4ms/step - loss: 2.6629 - acc: 0.1901 - val_loss: 2.5161 - val_acc: 0.2556
CPU times: user 3min 8s, sys: 1min 18s, total: 4min 26s
Wall time: 3min 41s


Evaluar la red en el conjunto de test, y mostramos el tiempo que le ha costado. 

In [23]:
%%time
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1),predictions.argmax(axis=1)))

[INFO] evaluating network...
             precision    recall  f1-score   support

          0       0.40      0.34      0.37      1000
          1       0.30      0.46      0.36      1000
          2       0.22      0.10      0.13      1000
          3       0.18      0.10      0.13      1000
          4       0.25      0.46      0.32      1000
          5       0.22      0.08      0.12      1000
          6       0.24      0.27      0.25      1000
          7       0.21      0.07      0.10      1000
          8       0.26      0.28      0.27      1000
          9       0.21      0.40      0.28      1000

avg / total       0.25      0.26      0.23     10000

CPU times: user 5.94 s, sys: 1.56 s, total: 7.5 s
Wall time: 16.3 s


# ShallowNet

Definimos la red

In [0]:
model = ShallowNet.build(width=32, height=32, depth=3, classes=10)

Compilamos la red.

In [0]:
opt = SGD(lr=0.005)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

Entrenamos la red

In [26]:
%%time
print("[INFO] training network...")
H = model.fit(trainX, trainY, validation_data=(testX, testY),batch_size=128, epochs=1, verbose=1)

[INFO] training network...
Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 10s 200us/step - loss: 14.5037 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
CPU times: user 7.71 s, sys: 2.03 s, total: 9.74 s
Wall time: 10.2 s


Evaluar la red en el conjunto de test, y mostramos el tiempo que le ha costado. 

In [27]:
%%time
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=128)
print(classification_report(testY.argmax(axis=1),predictions.argmax(axis=1)))

[INFO] evaluating network...
             precision    recall  f1-score   support

          0       0.00      0.00      0.00      1000
          1       0.00      0.00      0.00      1000
          2       0.00      0.00      0.00      1000
          3       0.00      0.00      0.00      1000
          4       0.00      0.00      0.00      1000
          5       0.00      0.00      0.00      1000
          6       0.00      0.00      0.00      1000
          7       0.10      1.00      0.18      1000
          8       0.00      0.00      0.00      1000
          9       0.00      0.00      0.00      1000

avg / total       0.01      0.10      0.02     10000

CPU times: user 1.33 s, sys: 294 ms, total: 1.63 s
Wall time: 1.63 s


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
